In [9]:
import sys
sys.path.append("..")

In [ ]:
from db.duckdb.duckdbhelper import DuckDBDatabaseHelper
from pprint import pprint

In [12]:
NUM_ROWS = 1500000

In [13]:
db = DuckDBDatabaseHelper("../meters.db")

In [14]:
sql_query ="""

SELECT day,energy_sum, 
temperatureMin, 
temperatureMax,
uvIndex,
dewPoint,                  
Type,
date_part('month', day) as month
FROM  meters m LEFT JOIN info_household ih
ON m.LCLid = ih.LCLid
LEFT JOIN weather_daily_darksky_modified wd
ON m.day = wd.formatted_temperatureMaxTime_weather_daily_darksky
LEFT JOIN uk_bank_holidays uk_bh
ON m.day = uk_bh."Bank holidays"

WHERE date_part('year', day) = 2013

"""

db.connect()
records = db.fetch_all(sql_query)
db.close_connection()

Connected to DuckDB database: ../meters.db
Fetched 1966727 rows.
Connection closed.


In [15]:
records[:5]

[(datetime.date(2013, 1, 3), 10.074, 9.65, 11.41, 1.0, 9.12, None, 1),
 (datetime.date(2013, 1, 6),
  10.293000000000003,
  6.91,
  8.61,
  1.0,
  7.0,
  None,
  1),
 (datetime.date(2013, 1, 7), 9.439000000000002, 7.1, 9.08, 1.0, 6.78, None, 1),
 (datetime.date(2013, 1, 11),
  10.978999900000002,
  0.89,
  5.7,
  1.0,
  1.68,
  None,
  1),
 (datetime.date(2013, 1, 12),
  10.585999900000003,
  1.75,
  4.53,
  1.0,
  0.79,
  None,
  1)]

In [16]:
COLS = [
"day","energy_sum", 
"temperatureMin", 
"temperatureMax",
"uvIndex",
"dewPoint",                  
"Type","month" ]

In [17]:
df_model = pd.DataFrame(records,columns = COLS)

In [18]:
df_model.head()

,day,energy_sum,temperatureMin,temperatureMax,uvIndex,dewPoint,Type,month
0,2013-01-03,10.074,9.65,11.41,1.0,9.12,None,1
1,2013-01-06,10.293,6.91,8.61,1.0,7.00,None,1
2,2013-01-07,9.439,7.10,9.08,1.0,6.78,None,1
3,2013-01-11,10.979,0.89,5.70,1.0,1.68,None,1
4,2013-01-12,10.586,1.75,4.53,1.0,0.79,None,1


In [19]:
df_model['Type_Binary'] = df_model['Type'].apply(lambda x: 1 if x is not None else 0)

In [20]:
COLS_FEAT = ["temperatureMin", 
"temperatureMax",
"uvIndex",
"dewPoint",                  
"Type_Binary","month"]
COLS_LABEL = ["energy_sum"]

In [21]:
df_model = df_model[COLS_FEAT + COLS_LABEL]

In [22]:
df_model = df_model.dropna()
len(df_model)

1940235

In [34]:
df_model.columns

Index(['temperatureMin', 'temperatureMax', 'uvIndex', 'dewPoint',
       'Type_Binary', 'month', 'energy_sum'],
      dtype='object')

In [35]:
if len(df_model) < NUM_ROWS:
    NUM_ROWS2 = len(df_model)
else:
    NUM_ROWS2 = NUM_ROWS

In [36]:
df_model_sample = df_model.sample(n = NUM_ROWS2)

In [37]:
X = df_model_sample[COLS_FEAT]

In [38]:
y = df_model_sample[COLS_LABEL]

In [39]:
X.head()

,temperatureMin,temperatureMax,uvIndex,dewPoint,Type_Binary,month
72494,8.02,9.61,2.0,8.38,0,3
1723302,1.16,7.65,1.0,1.32,0,2
1583410,1.75,4.53,1.0,0.79,0,1
739960,9.31,14.87,1.0,9.99,0,11
1743381,10.62,16.10,5.0,7.04,0,6


In [40]:
y

,energy_sum
72494,10.794
1723302,3.864
1583410,7.470
739960,23.909
1743381,6.312
...,...
1483398,20.253
542392,3.653
935471,16.690
8570,2.320


In [41]:
df_model_sample.to_csv("../model_data/train.csv",index = False)